In [ ]:
!pip install pymongo
!pip install requests
!pip install polars
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install h2o
!pip install fpdf2

SyntaxError: invalid syntax (3946235999.py, line 1)

In [ ]:
import mondongo
import cleanfiles
import eda


In [ ]:
mondongo.upload_data()

In [ ]:
eda.perform_eda()

In [ ]:
cleanfiles.clean_files()